## 一、数据爬取、整合及初步清洗

### 1. 网页爬虫、提取类

In [ ]:
'''
@Description: 个人爬虫及提取参考类
@Author: jkonel
@Date: 2020-06-18 09:15:14
@LastEditors: jkonel
@LastEditTime: 2020-06-22 17:13:10
'''

import requests
from lxml import etree
import pandas as pd
import re
from datetime import datetime
import time
import os
import chardet


class my_reptile:
    def __init__(self):
        pass
    
    ##------------------------SUB-TOOLS-------------------------##
    '''
    @description: 时间戳方法
    @param : self
    @return: 微秒，天，分钟
    '''
    def text_time(self):
        #[毫秒，日，分钟]
        tim = time.time()
        res = datetime.fromtimestamp(tim) 
        strc_exact = '{0}-{1}-{2}-{3}_{4}_{5}_{6}'.format(res.year, res.month, res.day, res.hour, res.minute, res.second, res.microsecond)
        strc_day = '{0}-{1}-{2}'.format(res.year, res.month, res.day)
        strc_min = '{0}-{1}-{2}-{3}_{4}'.format(res.year, res.month, res.day, res.hour, res.minute)
        return [strc_exact, strc_day, strc_min]
    
    '''
    @description: 字符串数字提取方法
    @param : stri
    @return: nums
    '''
    def get_nums(self, stri):
        res = re.findall(r'\d[?:\d\.]\d+?', stri)
        return float(res)#bug
    
    '''
    @description: 文件夹下所有文件名提取方法
    @param : folder
    @return: name list
    '''
    def get_files_name(self, folder):
        #print(list(os.walk(folder)))
        return list(os.walk(folder))[0][2]
    
    '''
    @description: 文件编码检测方法
    @param : filename
    @return: encoding_info
    '''
    def check_code(self, filename):
        bytes_data = open(filename,mode = 'rb').readline()
        encoding_info = chardet.detect(bytes_data)['encoding']
        return encoding_info
    ##--------------------SUB-TOOLS-END---------------------##
    
    '''
    @description: 网页爬虫下载方法
    @param : url:网址；name：保存名字
    @return: downloaded page
    '''
    def page_download(self, url, name):
        
        headers = {'user-agent': 'Mozilla/5.0'}
        path_name = './date_{0}'.format(self.text_time()[1])
        log_name = '{0}/log_{1}.txt'.format(path_name,self.text_time()[2])

        if not os.path.isdir(path_name):
            os.mkdir(path_name)
        if not os.path.isfile(log_name):
            with open(log_name,'w') as pp:
                pp.write('{0}/日志记录:{1}:\n'.format(path_name,self.text_time()[0]))

        respond = requests.get(url, headers=headers)

        if respond.status_code == 200:
            with open('{0}/{1}'.format(path_name,name),'w',encoding = respond.encoding) as fp:
                fp.write(respond.text)
            with open(log_name,'a') as pp:
                pp.write(self.text_time()[0] + '--{0}'.format(name) + '--写入成功！\n')
        else:
            print('读取失败！')

    #def html_read(self, file):
        ##path_name = './date_{0}'.format(self.text_time()[1])
        #html = etree.parse(file, parser = etree.HTMLParser(encoding = 'utf-8'))
        #return html
    
    '''
    @description: 网页内容提取写出方法 (写出文件名列表所有数据，数据提取以51jobs为例，便重载)
    @param:   folder name, html name, save csv name
    @return:  none
    @example: web_parsing('./demo',files_name_list,'csv_name.csv')
    '''
    def web_parsing(self,path_name,names_html,name_csv):
        dat = []
        for i in names_html:
            #name = 'page_1_{0}.html'.format(i)
            #path_name = './demo'  #文件夹名字demo,解析python网页
            try:
                with open(path_name +'/'+ i, 'r') as f:
                    c = f.read()
            except UnicodeDecodeError:
                with open(path_name +'/'+ i, 'r',encoding='utf-8') as f:
                    c = f.read()
            
            html = etree.HTML(c)
            
            items = html.xpath('//div[@class = "dw_table"]/div[@class = "el"]')
            #print(items)
            #-----------文件解析---------#
            for item in items:
                dic = {}
                
                title = item.xpath('.//p/span/a/@title')
                company = item.xpath('.//span[@class = "t2"]/a/text()')
                address = item.xpath('.//span[@class = "t3"]/text()')
                salery = item.xpath('.//span[@class = "t4"]/text()')
                date = item.xpath('.//span[@class = "t5"]/text()')
                
                if title:
                    title = title[0]
                else:
                    title = None

                if company:
                    company = company[0]
                else:
                    company = None

                if address:
                    address = address[0]
                else:
                    address = None

                if salery:
                    salery = salery[0]  #my_rep.get_nums(salery[0])
                else:
                    salery = None

                if date:
                    date = date[0]
                else:
                    date = None

                dic['title'] = title
                dic['company'] = company
                dic['address'] = address
                dic['salery'] = salery
                dic['date'] = date

                dat.append(dic)
        #--------文件保存---------#
        self.csv_save(dat, name_csv)

        
    '''
    @description: csv文件写出方法
    @param : csv data, csv name
    @return: none
    '''
    def csv_save(self, data, name):
        path_name = './date_{0}'.format(self.text_time()[1])
        if not os.path.isdir(path_name):
            os.mkdir(path_name)
        df = pd.DataFrame(data)
        df.to_csv('{0}/{1}'.format(path_name,name), index = 0, na_rep = 'NA')


### 2. 数据处理相关函数

In [ ]:
'''
@Description: 薪资统一为月薪
@Author: 李维强
@Date: 2020-06-23 
'''

import re
import pandas as pd
import  numpy as np

def sal(original_name,save_name):
    c=pd.read_csv(original_name)
    

    salarys = c['salery']
    reg_sa=r'\d+(?:\.\d+)?'
    
    print(salarys)

    salary_min = []
    salary_max =[]
    
    for sa in salarys:
        #print(sa)
        if sa is not np.nan :
            r=re.findall(reg_sa,sa)
            dict={ }
            if '万/月' in sa:
                sa_min=float(r[0])*10000
                sa_max=float(r[1])*10000
            elif '千/月' in sa:
                sa_min=float(r[0])*1000
                sa_max=float(r[1])*1000
            elif '万/年' in sa:
                sa_min=float(r[0])*10000//12
                sa_max=float(r[1])*10000//12
            elif '' in sa:
                sa_min=None
                sa_max=None
                
            salary_min.append(sa_min)
            salary_max.append(sa_max)
        else:
            salary_min.append(np.nan)
            salary_max.append(np.nan)
            continue
       
    c['salary_min'] = salary_min
    c['salary_max'] = salary_max
    
    df = pd.DataFrame(c)
    df.to_csv(save_name, index = 0, na_rep = 'NA')


#sal('.//all_page.csv','all_page_salery_normalize.csv')


In [ ]:
'''
@Description: 数据中城市映射到省份
@Author: Jkonel
@Date: 2020-06-23 09:46:57
@LastEditors: jkonel
@LastEditTime: 2020-06-24 20:10:20
'''

import re


'''
@description: 城市名提取函数
@param : city_name:输入原始字符串
@return: 处理后的字符串，只包含城市名(example: '上海市')
'''
def city_get(stri):
    res = re.findall(r'^[\u4e00-\u9fa5]+', stri)
    if res:
        if '异地' in res[0]:
            res = None
        else:
            res = res[0]+'市'
    else:
        res = None
    return res



'''
@description: 城市到省份映射函数
@param : 城市名 ('广州市')
@return: 省份/直辖市 ('广东省')
'''
def city2provence(city_name):
    dic_city2prov = {
        '北京市': '北京',
        '天津市': '天津',
        '石家庄市': '河北',
        '唐山市': '河北',
        '秦皇岛市': '河北',
        '邯郸市': '河北',
        '邢台市': '河北',
        '保定市': '河北',
        '张家口市': '河北',
        '承德市': '河北',
        '沧州市': '河北',
        '廊坊市': '河北',
        '衡水市': '河北',
        '太原市': '山西',
        '大同市': '山西',
        '阳泉市': '山西',
        '长治市': '山西',
        '晋城市': '山西',
        '朔州市': '山西',
        '晋中市': '山西',
        '运城市': '山西',
        '忻州市': '山西',
        '临汾市': '山西',
        '吕梁市': '山西',
        '呼和浩特市': '内蒙古',
        '包头市': '内蒙古',
        '乌海市': '内蒙古',
        '赤峰市': '内蒙古',
        '通辽市': '内蒙古',
        '鄂尔多斯市': '内蒙古',
        '呼伦贝尔市': '内蒙古',
        '巴彦淖尔市': '内蒙古',
        '乌兰察布市': '内蒙古',
        '兴安盟': '内蒙古',
        '锡林郭勒盟': '内蒙古',
        '阿拉善盟': '内蒙古',
        '沈阳市': '辽宁',
        '大连市': '辽宁',
        '鞍山市': '辽宁',
        '抚顺市': '辽宁',
        '本溪市': '辽宁',
        '丹东市': '辽宁',
        '锦州市': '辽宁',
        '营口市': '辽宁',
        '阜新市': '辽宁',
        '辽阳市': '辽宁',
        '盘锦市': '辽宁',
        '铁岭市': '辽宁',
        '朝阳市': '辽宁',
        '葫芦岛市': '辽宁',
        '长春市': '吉林',
        '吉林市': '吉林',
        '四平市': '吉林',
        '辽源市': '吉林',
        '通化市': '吉林',
        '白山市': '吉林',
        '松原市': '吉林',
        '白城市': '吉林',
        '延边朝鲜族自治州': '吉林',
        '哈尔滨市': '黑龙江',
        '齐齐哈尔市': '黑龙江',
        '鸡西市': '黑龙江',
        '鹤岗市': '黑龙江',
        '双鸭山市': '黑龙江',
        '大庆市': '黑龙江',
        '伊春市': '黑龙江',
        '佳木斯市': '黑龙江',
        '七台河市': '黑龙江',
        '牡丹江市': '黑龙江',
        '黑河市': '黑龙江',
        '绥化市': '黑龙江',
        '大兴安岭地区': '黑龙江',
        '上海市': '上海',
        '南京市': '江苏',
        '无锡市': '江苏',
        '徐州市': '江苏',
        '常州市': '江苏',
        '苏州市': '江苏',
        '南通市': '江苏',
        '连云港市': '江苏',
        '淮安市': '江苏',
        '盐城市': '江苏',
        '扬州市': '江苏',
        '镇江市': '江苏',
        '泰州市': '江苏',
        '宿迁市': '江苏',
        '杭州市': '浙江',
        '宁波市': '浙江',
        '温州市': '浙江',
        '嘉兴市': '浙江',
        '湖州市': '浙江',
        '绍兴市': '浙江',
        '金华市': '浙江',
        '衢州市': '浙江',
        '舟山市': '浙江',
        '台州市': '浙江',
        '丽水市': '浙江',
        '合肥市': '安徽',
        '芜湖市': '安徽',
        '蚌埠市': '安徽',
        '淮南市': '安徽',
        '马鞍山市': '安徽',
        '淮北市': '安徽',
        '铜陵市': '安徽',
        '安庆市': '安徽',
        '黄山市': '安徽',
        '滁州市': '安徽',
        '阜阳市': '安徽',
        '宿州市': '安徽',
        '巢湖市': '安徽',
        '六安市': '安徽',
        '亳州市': '安徽',
        '池州市': '安徽',
        '宣城市': '安徽',
        '福州市': '福建',
        '厦门市': '福建',
        '莆田市': '福建',
        '三明市': '福建',
        '泉州市': '福建',
        '漳州市': '福建',
        '南平市': '福建',
        '龙岩市': '福建',
        '宁德市': '福建',
        '南昌市': '江西',
        '景德镇市': '江西',
        '萍乡市': '江西',
        '九江市': '江西',
        '新余市': '江西',
        '鹰潭市': '江西',
        '赣州市': '江西',
        '吉安市': '江西',
        '宜春市': '江西',
        '抚州市': '江西',
        '上饶市': '江西',
        '济南市': '山东',
        '青岛市': '山东',
        '淄博市': '山东',
        '枣庄市': '山东',
        '东营市': '山东',
        '烟台市': '山东',
        '潍坊市': '山东',
        '济宁市': '山东',
        '泰安市': '山东',
        '威海市': '山东',
        '日照市': '山东',
        '莱芜市': '山东',
        '临沂市': '山东',
        '德州市': '山东',
        '聊城市': '山东',
        '滨州市': '山东',
        '菏泽市': '山东',
        '郑州市': '河南',
        '开封市': '河南',
        '洛阳市': '河南',
        '平顶山市': '河南',
        '安阳市': '河南',
        '鹤壁市': '河南',
        '新乡市': '河南',
        '焦作市': '河南',
        '濮阳市': '河南',
        '许昌市': '河南',
        '漯河市': '河南',
        '三门峡市': '河南',
        '南阳市': '河南',
        '商丘市': '河南',
        '信阳市': '河南',
        '周口市': '河南',
        '驻马店市': '河南',
        '武汉市': '湖北',
        '黄石市': '湖北',
        '十堰市': '湖北',
        '宜昌市': '湖北',
        '襄樊市': '湖北',
        '鄂州市': '湖北',
        '荆门市': '湖北',
        '孝感市': '湖北',
        '荆州市': '湖北',
        '黄冈市': '湖北',
        '咸宁市': '湖北',
        '随州市': '湖北',
        '恩施土家族苗族自治州': '湖北',
        '长沙市': '湖南',
        '株洲市': '湖南',
        '湘潭市': '湖南',
        '衡阳市': '湖南',
        '邵阳市': '湖南',
        '岳阳市': '湖南',
        '常德市': '湖南',
        '张家界市': '湖南',
        '益阳市': '湖南',
        '郴州市': '湖南',
        '永州市': '湖南',
        '怀化市': '湖南',
        '娄底市': '湖南',
        '湘西土家族苗族自治州': '湖南',
        '广州市': '广东',
        '韶关市': '广东',
        '深圳市': '广东',
        '珠海市': '广东',
        '汕头市': '广东',
        '佛山市': '广东',
        '江门市': '广东',
        '湛江市': '广东',
        '茂名市': '广东',
        '肇庆市': '广东',
        '惠州市': '广东',
        '梅州市': '广东',
        '汕尾市': '广东',
        '河源市': '广东',
        '阳江市': '广东',
        '清远市': '广东',
        '东莞市': '广东',
        '中山市': '广东',
        '潮州市': '广东',
        '揭阳市': '广东',
        '云浮市': '广东',
        '南宁市': '广西',
        '柳州市': '广西',
        '桂林市': '广西',
        '梧州市': '广西',
        '北海市': '广西',
        '防城港市': '广西',
        '钦州市': '广西',
        '贵港市': '广西',
        '玉林市': '广西',
        '百色市': '广西',
        '贺州市': '广西',
        '河池市': '广西',
        '来宾市': '广西',
        '崇左市': '广西',
        '海口市': '海南',
        '三亚市': '海南',
        '重庆市': '重庆',
        '成都市': '四川',
        '自贡市': '四川',
        '攀枝花市': '四川',
        '泸州市': '四川',
        '德阳市': '四川',
        '绵阳市': '四川',
        '广元市': '四川',
        '遂宁市': '四川',
        '内江市': '四川',
        '乐山市': '四川',
        '南充市': '四川',
        '眉山市': '四川',
        '宜宾市': '四川',
        '广安市': '四川',
        '达州市': '四川',
        '雅安市': '四川',
        '巴中市': '四川',
        '资阳市': '四川',
        '阿坝藏族羌族自治州': '四川',
        '甘孜藏族自治州': '四川',
        '凉山彝族自治州': '四川',
        '贵阳市': '贵州',
        '六盘水市': '贵州',
        '遵义市': '贵州',
        '安顺市': '贵州',
        '铜仁地区': '贵州',
        '黔西南布依族苗族自治州': '贵州',
        '毕节地区': '贵州',
        '黔东南苗族侗族自治州': '贵州',
        '黔南布依族苗族自治州': '贵州',
        '昆明市': '云南',
        '曲靖市': '云南',
        '玉溪市': '云南',
        '保山市': '云南',
        '昭通市': '云南',
        '丽江市': '云南',
        '普洱市': '云南',
        '临沧市': '云南',
        '楚雄彝族自治州': '云南',
        '红河哈尼族彝族自治州': '云南',
        '文山壮族苗族自治州': '云南',
        '西双版纳傣族自治州': '云南',
        '大理白族自治州': '云南',
        '德宏傣族景颇族自治州': '云南',
        '怒江傈僳族自治州': '云南',
        '迪庆藏族自治州': '云南',
        '拉萨市': '西藏',
        '昌都地区': '西藏',
        '山南地区': '西藏',
        '日喀则地区': '西藏',
        '那曲地区': '西藏',
        '阿里地区': '西藏',
        '林芝地区': '西藏',
        '西安市': '陕西',
        '铜川市': '陕西',
        '宝鸡市': '陕西',
        '咸阳市': '陕西',
        '渭南市': '陕西',
        '延安市': '陕西',
        '汉中市': '陕西',
        '榆林市': '陕西',
        '安康市': '陕西',
        '商洛市': '陕西',
        '兰州市': '甘肃',
        '嘉峪关市': '甘肃',
        '金昌市': '甘肃',
        '白银市': '甘肃',
        '天水市': '甘肃',
        '武威市': '甘肃',
        '张掖市': '甘肃',
        '平凉市': '甘肃',
        '酒泉市': '甘肃',
        '庆阳市': '甘肃',
        '定西市': '甘肃',
        '陇南市': '甘肃',
        '临夏回族自治州': '甘肃',
        '甘南藏族自治州': '甘肃',
        '西宁市': '青海',
        '海东地区': '青海',
        '海北藏族自治州': '青海',
        '黄南藏族自治州': '青海',
        '海南藏族自治州': '青海',
        '果洛藏族自治州': '青海',
        '玉树藏族自治州': '青海',
        '海西蒙古族藏族自治州': '青海',
        '银川市': '宁夏',
        '石嘴山市': '宁夏',
        '吴忠市': '宁夏',
        '固原市': '宁夏',
        '中卫市': '宁夏',
        '乌鲁木齐市': '新疆',
        '克拉玛依市': '新疆',
        '吐鲁番地区': '新疆',
        '哈密地区': '新疆',
        '昌吉回族自治州': '新疆',
        '博尔塔拉蒙古自治州': '新疆',
        '巴音郭楞蒙古自治州': '新疆',
        '阿克苏地区': '新疆',
        '克孜勒苏柯尔克孜自治州': '新疆',
        '喀什地区': '新疆',
        '和田地区': '新疆',
        '伊犁哈萨克自治州': '新疆',
        '塔城地区': '新疆',
        '阿勒泰地区': '新疆',
        '台湾省': '台湾',
        '香港特别行政区': '香港',
        '澳门特别行政区': '澳门'
    }
    
    provence=None
    for city in dic_city2prov:
        if city == city_name:
            provence = dic_city2prov[city]
    return provence


########------------Example----------------##########
#print(city2provence('杭州-滨江区'))

#prov = city2provence(city_get('杭州-滨江区'))

#print(prov)


In [ ]:
'''
@Description: 将每个地区转化为省份
@Author: 陈旭，季洪坤
@Date: 2020-06-23 
'''
from multLib.city2provence import city2provence
from multLib.city2provence import city_get
#from city2provence import city2provence
#from city2provence import city_get
import pandas as pd


def provence_add(original_name,save_name):    
    c=pd.read_csv(original_name)

    prov = c['address']
    provences=[]
    
    for addr in prov:   
        prove = city2provence(city_get(addr))
        provences.append(prove)

    c['provence'] = provences
    
    df = pd.DataFrame(c)
    df.to_csv(save_name, index = 0, na_rep = 'NA')

#provence_add('all_page_salery_normalize.csv','all_page_salery_normalize_add_provences.csv')


In [ ]:
'''
@Description: 将地区分为市和区
@Author: 李维强
@Date: 2020-06-23   11：08
'''

import pandas as pd
import numpy as np


def divide_city(original_name,save_name):
    b=pd.read_csv(original_name)
    
    city_area=b['address']
    
    city_=city_area.str.split('-').str.get(0)
    area_=city_area.str.split('-').str.get(1)

    b['city'] = city_
    b['area'] = area_
    
    df = pd.DataFrame(b)
    df.to_csv(save_name, index = 0, na_rep = 'NA')

#divide_city('all_page_salery_normalize_add_provences.csv','all_page_salery_normalize_add_provences_citys_areas.csv')


In [ ]:
'''
@Description: 删除空值数据
@Author: 李维强
@Date: 2020-06-23   12：02
'''
import pandas as pd

def delete_null(file_name, res_name):
    all_date = pd.read_csv(file_name)
    #not_null = all_date.dropna()    # 去空
    not_null = all_date.dropna(axis = 0,subset = ['title','company','address','salery','date','salary_min','salary_max','provence','city'])   # 丢弃指定列中有缺失值的行
    not_null.to_csv(res_name, index = 0)

#delete_null('./projects_multperson/all_page_salery_normalize_add_provences_citys_areas.csv','all_page_salery_normalize_add_provences_citys_areas_delete_null.csv')

In [ ]:
'''
@Description: csv文件筛选函数
@Author: Jkonel
@Date: 2020-06-24 15:00:33
@LastEditors: jkonel
@LastEditTime: 2020-06-24 18:19:20
'''

import  re
import  pandas as pd


'''
@description: csv文件筛选函数
@param : keyword:正则；original csv file；result csv file name
@return: none
'''
def data_filter(key_word, csv_file_name,save_name):
    data_frame = pd.read_csv(csv_file_name)

    rows = data_frame.shape[0]
    #print(rows)

    new_data = pd.DataFrame()
    for i in range(0, rows):
        a = data_frame.iloc[i,]
        title = a['title']
        judge = re.findall(key_word, title)
        if judge:
            new_data = new_data.append(a)
    new_data.to_csv(save_name, index=0, na_rep='NA')



#data_filter(r'嵌入式','./projects_multperson/date_2020-6-23/embeded_software_salary_norm_add_provence_city_area_del_none.csv','result_embeded_software.csv')


### 3. 数据爬取、整合、清洗

In [ ]:
'''
@Description: 网页爬取，提取，处理
@Author: Jkonel
@Date: 2020-06-22 17:23:11
@LastEditors: jkonel
@LastEditTime: 2020-06-23 16:24:33
'''

from multLib.my_reptile import my_reptile
from multLib.city_areas_add import city_areas_add
from multLib.provence_add import provence_add
from multLib.city2provence import city2provence
from multLib.city2provence import city_get
from multLib.delet_null import delet_null
from multLib.salary_normalize import salary_normalize
from multLib.data_filter import data_filter

my_rep = my_reptile()

###-----------------------------使用个人爬虫类爬取网页-----------------------------------###
#51jobs：python java c/c++ go php   linux 嵌入式软件 嵌入式硬件
# 修改不同搜索内容，得到全部网页数据

rang = 688 + 1

for i in range(1,rang):
    url = 'https://search.51job.com/list/000000,000000,0000,00,9,99,%25E5%25B5%258C%25E5%2585%25A5%25E5%25BC%258F%25E7%25A1%25AC%25E4%25BB%25B6%25E5%25B7%25A5%25E7%25A8%258B%25E5%25B8%2588,2,{0}.html?lang=c&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&ord_field=0&dibiaoid=0&line=&welfare='.format(i)
    name = 'page_8_{0}.html'.format(i)
    my_rep.page_download(url, name)

###-------------------------------python网页数据提取整合--------------------------------###
folder = './date_2020-6-19/php/'  #网页文件夹
files_list = my_rep.get_files_name(folder)


my_rep.web_parsing(folder,files_list,'all.csv')
for i in files_list:
    my_rep.web_parsing(folder,files_list,i.split('.')[0]+'prosess'+'.csv') ##后期有人工修正名字


###----------------------------------文件初步处理-----------------------------——---------###

folder = './date_2020-6-23/'
files_list = my_rep.get_files_name(folder)
#print(files_list)

for i in files_list:
    sal_norm = i.split('.')[0] + '_salary_norm.' + i.split('.')[1]
    prov_add = sal_norm.split('.')[0] + '_add_provence.' + sal_norm.split('.')[1]
    city_add = prov_add.split('.')[0] + '_city_area.' + prov_add.split('.')[1]
    del_none = city_add.split('.')[0] + '_del_none.' + city_add.split('.')[1]

    salary_normalize.sal(folder+i, sal_norm)          #将数据中的薪资规范化
    provence_add.provence_add(sal_norm, prov_add)     #将数据中的地址分析出省份
    city_areas_add.divide_city(prov_add, city_add)    #将数据中的地址提取出城市与地区
    delet_null.delete_null(city_add,del_none)         #初步去除垃圾数据


###----------------------------------文件进一步处理---------------------------——---------###

data_filter(r'嵌入式', './projects_multperson/date_2020-6-23/embeded_software_salary_norm_add_provence_city_area_del_none.csv','result_embeded_software.csv')
data_filter(r'嵌入式', './projects_multperson/date_2020-6-23/embeded_hardware_salary_norm_add_provence_city_area_del_none.csv','result_embeded_hardware.csv')
data_filter(r'(?:j|J)ava|JAVA', './projects_multperson/date_2020-6-23/java_salary_norm_add_provence_city_area_del_none.csv','result_java.csv')
data_filter(r'(?:P|p)ython|大数据','./projects_multperson/date_2020-6-23/python_salary_norm_add_provence_city_area_del_none.csv','result_python.csv')
data_filter(r'php|PHP|Php', './projects_multperson/date_2020-6-23/php_salary_norm_add_provence_city_area_del_none.csv','result_php.csv')
data_filter(r'go|Go|GO', './projects_multperson/date_2020-6-23/go_salary_norm_add_provence_city_area_del_none.csv','result_go.csv')
data_filter(r'(?:L|l)inux|LINUX', './projects_multperson/date_2020-6-23/linux_salary_norm_add_provence_city_area_del_none.csv','result_linux.csv')
data_filter(r'(?:C|c)\+\+|c|C', './projects_multperson/date_2020-6-23/cc++_salary_norm_add_provence_city_area_del_none.csv','result_cc++.csv')



## 二、数据分析、展示

> 相关函数 

    地图封装函数注意:pyecharts版本：1.7.0

In [96]:
'''
@Description: 地图封装整合 (便于将多个职业的数据在同一个地图显示)
@Author: Jkonel
@Date: 2020-06-23 20:45:42
@LastEditors: jkonel
@LastEditTime: 2020-06-24 16:50:39
'''

from pyecharts.charts import Map
from typing import List
import pyecharts.options as opts
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Timeline, Grid, Bar, Map, Pie, Line


##原始数据结构例子
''''''
data_example = [
    {
        'job':'java',
        'data': [
            {"province": "广东", "value": 2222},
            #.............#
            {"province": "西藏", "value": 3000}
            ]
    },
    {
        'job':'python',
        'data': [
            {"province": "山东", "value": 5000},
            #.............#
            {"province": "西藏", "value": 3000}
            ]
    },
]



'''
@description: 得到名字列表特定名字下标
@param : list,name
@return: index
'''
def get_list_index(list, str):
    index = 0
    for stri in list:
        if stri is str:
            return index
        else:
            index += 1
    return None

'''
@description: 
@param : job_name:'java';a:规范化数据;name:规范化名字列表
@return: grid_chart
'''
def get_chart(job_name, a, name):
    index = get_list_index(name, job_name)
    da = a[index]
    maxNum = max([x[1] for x in da])
    minNum = min([x[1] for x in da])

    map_chart = (
        Map()
        .add(
            series_name="",
            data_pair=da,
            zoom=1,
            center=[119.5, 34.5],
            is_map_symbol_show=False,
            itemstyle_opts={
                "normal": {"areaColor": "#323c48", "borderColor": "#404a59"},
                "emphasis": {
                    "label": {"show": Timeline},
                    "areaColor": "rgba(255,255,255, 0.5)",
                },
            },
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title = job_name,###
                subtitle="",
                pos_left="center",
                pos_top="top",
                title_textstyle_opts=opts.TextStyleOpts(
                    font_size=25, color="rgba(255,255,255, 0.9)"
                ),
            ),
            tooltip_opts=opts.TooltipOpts(
                is_show=True,
                formatter=JsCode(
                    """function(params) {
                    if ('value' in params.data) {
                        return params.data.value[2] + ': ' + params.data.value[0];
                    }
                }"""
                ),
            ),
            visualmap_opts=opts.VisualMapOpts(
                is_calculable=True,
                dimension=0,
                pos_left="30",
                pos_top="center",
                range_text=["High", "Low"],
                range_color=["lightskyblue", "yellow", "orangered"],
                textstyle_opts=opts.TextStyleOpts(color="#ddd"),
                min_=10,
                max_=2000,
            ),
        )
    )
    bar_x_data = [x[0] for x in da]
    bar_y_data = [{"name": x[0], "value": x[1]} for x in da]
    bar = (
        Bar()
        .add_xaxis(xaxis_data=bar_x_data)
        .add_yaxis(
            series_name="",#job_name,##
            yaxis_data=bar_y_data,
            label_opts=opts.LabelOpts(
                is_show=True, position="right", formatter="{b} : {c}"
            ),
        )
        .reversal_axis()
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                max_=maxNum, axislabel_opts=opts.LabelOpts(is_show=False)
            ),
            yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(is_show=False)),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            visualmap_opts=opts.VisualMapOpts(
                is_calculable=True,
                dimension=0,
                pos_left="10",
                pos_top="top",
                range_text=["High", "Low"],
                range_color=["lightskyblue", "yellow", "orangered"],
                textstyle_opts=opts.TextStyleOpts(color="#ddd"),
                min_=minNum,
                max_=maxNum,
            ),
        )
    )

    pie = (
        Pie()
        .add(
            series_name = "",#job_name,##
            data_pair = da,###
            radius=["15%", "35%"],
            center=["80%", "70%"],
            itemstyle_opts=opts.ItemStyleOpts(
                border_width=1, border_color="rgba(0,0,0,0.3)"
            ),
        )
        .set_global_opts(
            tooltip_opts=opts.TooltipOpts(is_show=True, formatter="{b} {d}%"),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )
    
    grid_chart = (
        Grid()
            .add(
                bar,
                grid_opts=opts.GridOpts(
                    pos_left="10", pos_right="95%", pos_top="40%", pos_bottom="5"
                ),
            )
        .add(map_chart, grid_opts=opts.GridOpts())
        .add(pie, grid_opts=opts.GridOpts(pos_left="45%", pos_top="50%"))
    )
    return grid_chart


'''
@description: 地图主输出函数
@param : 规范原始数据
@return: 地图输出
'''
def multicolorful_map(data):
    #有多少name就有多少个a元素
    name = [x['job'] for x in data]
    a = [ [[x['province'],x['value']] for x in d['data'] ] for d in data]
    #a =[[x['province'],x['value']] for x in data]

    timeline = Timeline(
        init_opts=opts.InitOpts(width="960px", height="480px", theme=ThemeType.DARK)
    )
    indexs = 0
    for job_name in name:
        timeline.add(get_chart(job_name, a, name), time_point=indexs)
        indexs+=1

    timeline.add_schema(
        orient="vertical",
        is_auto_play=True,
        is_inverse=True,
        play_interval=5000,
        pos_left="null",
        pos_right="5",
        pos_top="20",
        pos_bottom="20",
        width="60",
        label_opts=opts.LabelOpts(is_show=True, color="#fff"),
    )
    return timeline

##-------------EXAMPLE---------------##
# timelines = multicolorful_map(data)
# timelines.render_notebook()


### 1.地区对招聘数量、 招聘薪资的影响

####         (0) 职业取均值， 不同地区招聘数量、 薪资分布

In [99]:
'''
@Description: 省总岗位数（全部）
@Author: 李维强
@Date: 2020-06-25
'''
import pandas as pd
import numpy as np
#from multicolorful_map import multicolorful_map

data_example = [
    {
        'job':'java',
        'data': [
            {"province": "广东", "value": 2222},
            #.............#
            {"province": "西藏", "value": 3000}
            ]
    },
    {
        'job':'python',
        'data': [
            {"province": "山东", "value": 5000},
            #.............#
            {"province": "西藏", "value": 3000}
            ]
    },
]


folder = './date_2020-6-23/all'
name =folder+'/all.csv'
save_name = folder + '/all_salary_mean.csv'

def sal_mean(original_name,save_name):
    c=pd.read_csv(original_name)
    
    c['salary_mean'] = c[['salary_min','salary_max']].mean(axis=1)
    #c.loc[:, 'salary_mean'] = np.mean([c['salary_min'], c['salary_max']],0)


    df = pd.DataFrame(c)
    df.to_csv(save_name, index = 0, na_rep = 'NA')
    

sal_mean(name,save_name)


all_date1=pd.read_csv(name)
df_all_date1=all_date1.groupby('province').count()['title']

all_date2=pd.read_csv(save_name)
df_all_date2=all_date2.groupby('province').mean()['salary_mean'].astype(int)


list1_null=[]
list2_null=[]


province_index1=list(df_all_date1.index)
province_value1=list(df_all_date1.values)

province_index2=list(df_all_date2.index)
province_value2=list(df_all_date2.values)


for i in range(len(province_index1)):
    dict1_null={}
    dict2_null={}
    
    dict1_null['province']=province_index1[i]  
    
    dict1_null['value']=int(province_value1[i])
    list1_null.append(dict1_null.copy())

    dict2_null['province']=province_index2[i]  
    dict2_null['value']=int(province_value2[i])    
    list2_null.append(dict2_null.copy())

print(type(data_example[0]['data'][0]['value']))

data=[]
list1_date={}
list1_date["job"]="不同地区招聘数量分布"
list1_date["data"]=list1_null
list2_date={}
list2_date["job"]="不同地区招聘薪资分布"
list2_date["data"]=list2_null


data.append(list1_date)
data.append(list2_date)

timelines = multicolorful_map(data)
timelines.render_notebook()

<class 'int'>


#### (1) 对比各个职业，不同地域的招聘数量、 薪资分布

In [98]:
'''
@Description: 同一职业，不同地域的薪资分布
@Author: 陈旭
@Date: 2020-06-24
@LastEditors: jkonel
@LastEditTime: 2020-06-23 16:24:33
'''

import pandas as pd
import numpy as np
import math 
import numpy as np 
import os
#from multLib.multiColorful_map import multicolorful_map

provinet_list =[
     '湖北',
     '陕西',
     '青海',
     '广东',
     '辽宁',
     '宁夏',
     '江西',
     '上海',
     '山西',
     '甘肃',
     '香港',
     '内蒙古',
     '河南',
     '广西',
     '澳门',
     '北京',
     '云南',
     '海南',
     '河北',
     '四川',
     '西藏',
     '吉林',
     '江苏',
     '天津',
     '重庆',
     '台湾',
     '贵州',
     '新疆',
     '山东',
     '福建',
     '浙江',
     '黑龙江',
     '安徽',
     '湖南'
]

folder = './date_2020-6-23'

file_list = os.walk(folder)
files_name = [x for x in file_list][0][2]

result_data_q1 = []

for fil in files_name:
    dic = {}
    dic['job'] = fil.split('.')[0]
    
    single_jobs = pd.read_csv(folder+'/'+fil)
    
    sub_data = []
    for i in provinet_list:
        sub_dic = {}
        sub_dic['province'] = i
        
        res = single_jobs.loc[single_jobs['province']==i]
        res_mean_min = res['salary_min'].mean()
        res_mean_max = res['salary_max'].mean()
        res_mean = round((res_mean_min+res_mean_max)/2,2)
        sub_dic['value'] = res_mean
        sub_data.append(sub_dic)
        
    dic['data'] = sub_data
    result_data_q1.append(dic)
#print(result_data_q1)

timelines = multicolorful_map(result_data_q1)
timelines.render_notebook()


## 2.职业方向对招聘数量、 招聘薪资的影响

#### （2）地域取数学期望， 不同职业的薪资对比

In [48]:
'''
@Description: 全部地域一起考虑，不同职业 的平均招聘薪资的对比(分析2)
@Author: 陈旭
@Date: 2020-06-24
@LastEditors: 
@LastEditTime:
'''

import pandas as pd
import numpy as np
import math 
import numpy as np 
import os


from pyecharts import options as opts
from pyecharts.charts import Pie
from pyecharts.faker import Faker



python_jobs=pd.read_csv('date_2020-6-23/result_python.csv')
n2=round((python_jobs['salary_min'].mean()+python_jobs['salary_min'].mean())/2,2) #地域取均值，不同职业的薪资对比
print('python:',n2) 

php_jobs=pd.read_csv('date_2020-6-23/result_php.csv')
n3=round((php_jobs['salary_min'].mean()+php_jobs['salary_min'].mean())/2,2)
print('php:',n3)

linux_jobs=pd.read_csv('date_2020-6-23/result_linux.csv')
n4=round((linux_jobs['salary_min'].mean()+linux_jobs['salary_min'].mean())/2,2)
print('linux:',n4)

java_jobs=pd.read_csv('date_2020-6-23/result_java.csv')
n5=round((java_jobs['salary_min'].mean()+java_jobs['salary_min'].mean())/2,2)
print('java:',n5)

go_jobs=pd.read_csv('date_2020-6-23/result_go.csv')
n6=round((go_jobs['salary_min'].mean()+go_jobs['salary_min'].mean())/2,2)
print('go:',n6)

embeded_software_jobs=pd.read_csv('date_2020-6-23/result_embeded_software.csv')
n7=round((embeded_software_jobs['salary_min'].mean()+embeded_software_jobs['salary_min'].mean())/2,2)
print('embeded_software:',n7)

embeded_hardware_jobs=pd.read_csv('date_2020-6-23/result_embeded_hardware.csv')
n8=round((embeded_hardware_jobs['salary_min'].mean()+embeded_hardware_jobs['salary_min'].mean())/2,2)
print('embeded_hardware:',n8)

cc_jobs=pd.read_csv('date_2020-6-23/result_cc++.csv')
n9=(cc_jobs['salary_min'].mean()+cc_jobs['salary_min'].mean())/2
print('c/c++:',n9)


#---------show---------#
name=['Python','Php','Linux','Java','Go','embeded_software','embeded_hardware','c/c++']
mean_values=[n2,n3,n4,n5,n6,n7,n8,n9]

c = (
    Pie()
    .add("", [list(z) for z in zip(name, mean_values)])
    
    .set_global_opts(title_opts=opts.TitleOpts(title="不同职业薪资均值对比"),
    legend_opts=opts.LegendOpts(
                type_="scroll", pos_top="90%", pos_left="70%", orient="horizontal"
    ), 
                    )
                
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}/ {c}"))
    
)
c.render_notebook()


python: 11980.71
php: 9301.26
linux: 9508.98
java: 10787.02
go: 13339.29
embeded_software: 10398.71
embeded_hardware: 9190.86
c/c++: 12003.740510777881


#### (3) 薪资取数学期望， 不同职业的数量对比

In [33]:
'''
@Description: 薪资取数学期望， 不同职业的数量对比
@Author: 薛逸凡
@Date: 2020-06-25
@LastEditors: 
@LastEditTime:
'''


import pandas as pd
import numpy as np

folder = './date_2020-6-23/'

cc=pd.read_csv(folder+'result_cc++.csv')
hard=pd.read_csv(folder+'result_embeded_hardware.csv')
soft=pd.read_csv(folder+'result_embeded_software.csv')
go=pd.read_csv(folder+'result_go.csv')
java=pd.read_csv(folder+'result_java.csv')
linux=pd.read_csv(folder+'result_linux.csv')
php=pd.read_csv(folder+'result_php.csv')
python=pd.read_csv(folder+'result_python.csv')
jobs=[
    {"c/c++工程师":(len(cc))},
    {"嵌入式硬件工程师":(len(hard))},
    {"嵌入式软件工程师":(len(soft))},
    {"GO开发工程师":(len(go))},
    {"JAVA开发工程师":(len(java))},
    {"Linux开发工程师":(len(linux))},
    {"PHP开发工程师":(len(php))},
    {"python工程师":(len(python))}
    ]
jobs


[{'c/c++工程师': 8536},
 {'嵌入式硬件工程师': 857},
 {'嵌入式软件工程师': 6499},
 {'GO开发工程师': 502},
 {'JAVA开发工程师': 24689},
 {'Linux开发工程师': 7370},
 {'PHP开发工程师': 3033},
 {'python工程师': 3109}]

In [47]:
'''
@Description: 薪资取数学期望， 不同职业的数量对比
@Author: 薛逸凡
@Date: 2020-06-25
@LastEditors: 
@LastEditTime:
'''

from pyecharts import options as opts
from pyecharts.charts import Bar

c = (
    Bar()
    .add_xaxis(
        [
            "全部职业",
        
        ]
    )
    .add_yaxis("c/c++", [8536],color='blue')
    .add_yaxis("嵌入式硬件开发", [857],color='green')
    .add_yaxis("嵌入式软件开发", [6499],color='red')
    .add_yaxis("GO", [502],color='black')
    .add_yaxis("JAVA", [24689],color='pink')
    .add_yaxis("Linux", [7370],color='yellow')    
    .add_yaxis("PHP", [3033],color='orange')
    .add_yaxis("python", [3109],color='purple')
    
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=0)),
        title_opts=opts.TitleOpts(title="此职业岗位总数"),
    )
   
)

c.render_notebook()